In [ ]:
import os
import tensorflow as tf
from PIL import Image

cwd = os.getcwd()  #获取当前目录
classes = {'bus','flower','long','horse','elephant'}  #预先定义自己要分的类别

##########################制作tfrecord数据
writer = tf.python_io.TFRecordWriter('fenlei_train.tfrecords')  #要生成的TFRecord的名字

def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list= tf.train.BytesList(value=[value]))

for index, name in enumerate(classes):  #第一个循环是打开已经预先定义好的五个文件夹
    class_path = cwd+'/'+name+'/'
    
    for img_name in os.listdir(class_path):  #开始遍历每一张图片
        img_path = class_path+img_name  #每个图片的地址
        
        img = Image.open(img_path)  #打开每一张图片
        img = img.resize((64,64))  #将图片设置为指定的大小
        img_raw = img.tobytes() #将图片转化为二进制格式
        
          ######### 这里的内容就是对label image进行封装
        example = tf.train.Example(
            features=tf.train.Features(feature={
                'label':_int64_feature(index),
                'img_raw':_bytes_feature(img_raw)
                }))
          #########
            
        writer.write(example.SerializeToString())#序列化为字符串
writer.close()

            
    
    
    


#定义读取TFRecord的数据函数
def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename])  #生成一个queue队列
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue) #返回文件名和文件

        #解析label和image信息，这个名字需要与生成TFRecord的时候一致
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label':tf.FixedLenFeature([],tf.int64),
                                           'img_raw':tf.FixedLenFeature([],tf.string)
                                            }) #将image数据个label取出来
    img = tf.decode_raw(features['img_raw'],tf.uint8)
    img = tf.reshape(img,[64,64,3]) #reshape为64*64的3通道照片
    label = tf.cast(features['label'],tf.int32) #tf.cast就是进行数据转换为int32，在数据流中抛出label张量
    return img,label

############其实就是到这里就可以把数据导入到神经网络中去， xs,ys = read_and_decode(filename)




#保存以及显示图片
gen_pic = '/home/ginchoeng/jupyternotebook/yunxing/gen_pic'#获取当前的运行路径 ,这里就是把生成标签的图片保存的路径
init = tf.global_variables_initializer()
batch = read_and_decode('fenlei_train.tfrecords')

with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads =tf.train.start_queue_runners(coord=coord)

    for i in range(400):
        example, lab = sess.run(batch)
        img = Image.fromarray(example,'RGB')
        img.save(gen_pic+'/'+str(i)+'_label_'+str(lab)+'.jpg')
        print(example,lab)
        coord.request_stop()
        coord.join(threads)


In [ ]:
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data  #导入数据
#下面这一行跟分类器中的是一样的
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

time_start = time.time()

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) # shape表示生成张量的维度 stddev是标准差
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#定义卷积神经网络层了
def conv2d(x,W):   # x代表输入的值，或者是图片的值。 W就是weight
    #strides=[1,x轴移动距离，y轴移动距离，1]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')#padding就是选择是否使用填充的扫描方式
    #这里就是返回tensorflow中2维的卷积神经网络。 
    
    #第三个参数就是设置步长，在tensorflow中它是一个长度为4的列表。在这个列表中第一个以及最后一个元素都是必须等于1的。列表中间的两个
    #参数分别是代表在x轴，y轴上移动的距离
    
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #使用最大池化的方法， ksize=[1,height,width,1]这个是池化窗口的大小

#设置placeholder来进行数据的传输
xs = tf.placeholder(tf.float32,[None,4096])
ys = tf.placeholder(tf.float32,[None,5])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs,[-1,64,64,3]) #在定义卷积层之前，先要定义一下我们的输入。就是图片的输入
                                      #-1表示不规定样本数量的多少，28×28是代表这个图片的尺寸，最后的1 是代表深度为1.
      
#第一层卷积层
W_conv1 = weight_variable([5,5,3,32]) # 5*5就是patch的大小(就是卷积核的大小),1是代表输入通道数(如果是彩色图像就是3),32是代表有32个神经元就是有32个卷积核去关注32个特征 
b_conv1 = bias_variable([32]) #有32个权值，那么所以就对应有32个偏置。
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1) #这一行就是卷积行    输出的大小是28*28*32,因为使用的padding是same，所以长宽不变
h_pool1 = max_pool_2x2(h_conv1) #这一行的输出就是这一层卷积池化之后得到的结果   输出的大小是14*14*32,因为扫描的间距是2，所以缩小了一半

#第二层卷积层
W_conv2 = weight_variable([5,5,32,64]) #32是上一层的输出通道数，变成下一层的输入通道数。然后用64个卷积核去patch
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2) #输出的大小14*14*64
h_pool2 = max_pool_2x2(h_conv2)                          #输出的大小7*7*64

#全链接层1
W_fc1 = weight_variable([7*7*64,1024]) #输入就是上一层的输出，使用包含1024个神经元的一层来处理整个图片
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])#把第二层卷积网络出来的 三维数据转换为一维数据
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#全链接层2
W_fc2 = weight_variable([1024,10]) #上一层的输出就是下一层的输入，最终是判断出0-9的图片
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)  #最后一层的分类时使用的激活函数使用softmax
# h_fc2_drop = tf.dropout(h_fc2,keep_prob)  #输出层可以不使用dropout


#设置代价误差函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
# tf.summary.scalar('loss',cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)#不采用梯度下降的原因是：庞大的系统应该使用更好的优化器

#全局变量的初始化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(2000):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.5})
    if i % 50 == 0:
        print(compute_accuracy(mnist.test.images[:1000],mnist.test.labels[:1000]))

        
time_end = time.time()
print('使用的时间：',time_end-time_start,'s')

In [1]:
print(64*64)

4096
